# Quantum Computing -based Optimization for Sustainable Data Workflows in Cloud Infrastructures

by [Valter Uotila](https://researchportal.helsinki.fi/en/persons/valter-johan-edvard-uotila), PhD student, [Unified Database Management Systems](https://www2.helsinki.fi/en/researchgroups/unified-database-management-systems-udbms/news), University of Helsinki

This is just a specified shortest path finding application applied to the problem presented in the [document](https://github.com/valterUo/Quantum-Computing-based-Optimization-for-Sustainable-Data-Workflows-in-Cloud/blob/main/Quantum_Computing__based_Optimization_for_Sustainable_Data_Workflows_in_Cloud.pdf) that comes along with this implementation.

Possible quantum software-harware combinations to solve the problem:

1. Amazon Braket's D-Wave — Advantage and D-Wave — 2000Q
    1. Ocean implementation of this code
2. Amazon Braket's IonQ and Rigetti machines
    1. Qiskit implementation of this code
3. Amazon Braket's simulators
    1. Qiskit implementation of this code
4. D-wave's Leap Advantage
    1. Ocean implementation of this code
5. IBM Quantum systems
    1. Qiskit implementation of this code
6. Local machine
    1. Both Ocean and Qiskit versions and also the non-quantum version of the algorithm

Because I am familiar with the Ocean framework and it is specially designed for formulating QUBOs, I initially formulated the problem using it.

## Chapter 1: Implementation using Ocean connecting to Amazon Braket quantum annealers or D-wave Leap quantum annealers

In [1]:
import dimod
from dimod.generators.constraints import combinations
from dwave.system import LeapHybridSampler
from hybrid.reference import KerberosSampler
from dwave.system.composites import EmbeddingComposite

from braket.aws import AwsDevice
from braket.ocean_plugin import BraketSampler, BraketDWaveSampler

import numpy as np
import json
import itertools
import os
import math
import random
import networkx as nx
import matplotlib.pyplot as plt

notebook_path = os.path.abspath("main.ipynb")

In [2]:
def append_linear_safe(variable, value, linear_dict):
    if variable in linear_dict.keys():
        linear_dict[variable] = linear_dict[variable] + value
    else:
        linear_dict[variable] = value

def append_quadratic_safe(variable, value, quadratic_dict):
    if variable in quadratic_dict.keys():
        quadratic_dict[variable] = quadratic_dict[variable] + value
    else:
        quadratic_dict[variable] = value

## Importing data

In [3]:
cloud_partners_file_path = os.path.join(os.path.dirname(notebook_path), "data/cloud_partners.json")
f = open(cloud_partners_file_path)
partners_root = json.load(f)
cloud_partners = partners_root["cloud_partners"]

workload_name = "workload1.json"
workload_file_path = os.path.join(os.path.dirname(notebook_path), "data/workloads/" + workload_name)
f = open(workload_file_path)
workload_root = json.load(f)
workload = workload_root["workload"]

#print(cloud_partners)
#print(workload)

## Emission simulator

This section implements an emission simulator which simulates emission changes in data center operations. Note that it is relatively hard to get accurate data from individual data centers. This simulator is just for demonstration and it does not have an actual scientific background.

In [4]:
def emission_simulator(variable1, variable2, cloud_partners, workload):
    simulated_carbon_footprint = 1
    emission_factor = 1
    workload_type_in_process = None
    
    source_data_center_id = variable1[1]
    work_in_process = variable2[0]
    target_data_center_id = variable2[1]
    
    for work in workload:
        if work["work_id"] == int(work_in_process):
            emission_factor = work["emission_factor"]
            workload_type_in_process = work["work_type"]
    
    for partner in cloud_partners:
        for center in partner["data_centers"]:
            if target_data_center_id == center["center_id"]:
                for workload_type in center["workload_dependent_emissions"]:
                    if workload_type_in_process == workload_type["workload_type"]:
                        interval = workload_type["emission_interval"]
                        #print(interval)
                        simulated_carbon_footprint = emission_factor*random.uniform(interval[0], interval[1])
            
    return simulated_carbon_footprint

## Creating variables for the binary quadratic model

We defined variables to be $ x_{i,j} = (w_i, d_j) $.

In [5]:
# We assume that any work can be executed on any data center
def construct_variables(start_id):
    variables = dict()
    workload_order = []
    for work in workload[start_id:]:
        variables[str(work["work_id"])] = list()
        workload_order.append(str(work["work_id"]))
        for partner in cloud_partners:
            for center in partner["data_centers"]:
                # The each key in the variables dictionary corresponds to a level in a tree i.e. a time step in the workflow
                variables[str(work["work_id"])].append((str(work["work_id"]), center["center_id"]))
    return variables, workload_order
            
#print(json.dumps(variables, indent=1))

## Constructing constraints 

### Constraint 1

This constraint implements the requirement that for every work $ w_i $ we have exactly one variable $ x_{i,j} = (w_i, d_j) = 1$. In other words, this means that every work is executed on a single data center.

In [6]:
def construct_bqm_constraint1(bqm, variables):
    strength = 25.0
    for work_id in variables:
        one_work_bqm = combinations(variables[work_id], 1, strength=strength)
        bqm.update(one_work_bqm)
    return bqm

### Constraint 2

This constraint implements the requirement that for every pair of variables $x_{i,j} = (w_i, d_j)$ and $x_{i+1,k} = (w_{i+1}, d_k)$ we associate the (estimated emission) coefficient $e(x_{i,j}, x_{i+1,k})$. This coefficient is calculated in emission_simulator function. Note that we need to calculate this only for those pairs, where the works $w_i$ and $w_{i+1}$ are consecutive works in the workload.

To evaluate the algorithm we store the tree in a networkx graph.

In [7]:
def construct_bqm_constraint2(bqm, variables, workload_order):
    vartype = dimod.BINARY
    A = 1
    linear = dict()
    quadratic = dict()
    offset = 0.0
    tree = nx.Graph()

    for work_id_current in range(len(workload_order) - 1):
        work_id_next = work_id_current + 1
        key_current = workload_order[work_id_current]
        key_next = workload_order[work_id_next]

        for work1 in variables[key_current]:
            for work2 in variables[key_next]:
                coeff = emission_simulator(work1, work2, cloud_partners, workload)
                append_quadratic_safe((work1, work2), coeff, quadratic)
                tree.add_edge(work1, work2, weight=coeff)

                #print("Works", work1, work2)
                #print("Coefficient", coeff)

    bqm_c2 = dimod.BinaryQuadraticModel(linear, quadratic, offset, vartype)
    bqm_c2.scale(A)
    bqm.update(bqm_c2)
    return bqm, tree

#print(bqm)
#print(bqm.to_numpy_vectors())

## Demonstrating algorithm

In [8]:
def compare_to_optimal(solution, tree, optimal_weight):
    current_total = 0
    try:
        for i in range(len(solution) - 1):
            edge_weight = tree.get_edge_data(solution[i], solution[i+1])
            current_total += edge_weight["weight"]
    except:
      print("The quantum result contains edges which are not in the tree.")
    return np.abs(optimal_weight - current_total)/optimal_weight

In [9]:
def print_solution(sample, tree, optimal_weight = -1):
    positive_solution = []
    for varname, value in sample.items():
        if value == 1:
            positive_solution.append(varname)
            print(varname, value)
    if optimal_weight != -1:
        print("Difference from the optimal ", compare_to_optimal(positive_solution, tree, optimal_weight))

### Wrapping up various methods to solve the QUBO

In [10]:
def solve_bqm_in_leap(bqm, sampler = "Kerberos"):
    bqm.normalize()
    if sampler == "Kerberos":
        kerberos_sampler = KerberosSampler().sample(bqm, max_iter=10, convergence=3, qpu_params={'label': 'Data workflow optimization'})
        sample = kerberos_sampler.first.sample
    elif sampler == "LeapHybrid":
        sampler = LeapHybridSampler()
        sampleset = sampler.sample(bqm)
        sample = sampleset.first.sample
    return sample
    
    #print(sampleset)
    #print(best_solution)
    #sample = best_solution
    #energy = sampleset.first.energy

In [11]:
def solve_bqm_in_amazon_braket(bqm, system = "Advantage"):
    device = None
    num_reads = 1000
    if system == "Advantage":
        device = "arn:aws:braket:::device/qpu/d-wave/Advantage_system4"
    elif system == "2000Q":
        device = "arn:aws:braket:::device/qpu/d-wave/DW_2000Q_6"
    sampler = BraketDWaveSampler(device_arn = device)
    sampler = EmbeddingComposite(sampler)
    sampleset = sampler.sample(bqm, num_reads=num_reads)
    sample = sampleset.first.sample
    return sample

In [12]:
def solve_with_simulated_annealing(bqm):
    sampler = dimod.SimulatedAnnealingSampler()
    sampleset = sampler.sample(bqm, num_reads=100)
    sample = sampleset.first.sample
    return sample

In [13]:
def solve_exactly(bqm):
    sampler = dimod.ExactSolver()
    sampleset = sampler.sample(bqm)
    sample = sampleset.first.sample
    return sample

In [14]:
def solve_with_networkx(tree, variables, start_work):
    possible_solutions = []
    best_solution = None
    min_weight = float('Inf')
    for source_var in variables[start_work]:
        for target_var in variables['5']:
            possible_solutions.append(nx.dijkstra_path(tree, source=source_var, target=target_var))
    for sol in possible_solutions:
        current_total = 0
        for i in range(len(sol) - 1):
            edge_weight = tree.get_edge_data(sol[i], sol[i+1])
            current_total += edge_weight["weight"]
        #print("Shortest path ", sol)
        #print("Current total ", current_total)
        if min_weight > current_total:
            min_weight = current_total
            best_solution = sol
    return best_solution, min_weight

### Run single time step (for test and demonstration purposes)

In [15]:
# vartype = dimod.BINARY
# bqm = dimod.BinaryQuadraticModel({}, {}, 0.0, vartype)
# variables = construct_variables(0)
# bqm = construct_bqm_constraint1(bqm)
# bqm, tree = construct_bqm_constraint2(bqm)
# print(bqm)

#print("The problem is to find the minimum path from some of the nodes ('0', x) to some of the nodes ('5', y). The weight of the edges are defined by carbon footprint associated to the computation.")
#nx.draw(tree, with_labels = True)

#### Optimal and correct solution for evaluation

In [16]:
#best_solution, optimal_weight = solve_with_networkx(tree, '0')
#print(best_solution, optimal_weight)

The following results we obtain with annealing. Ideally we would be close to the results we obtain from the function solve_with_networkx.

In [17]:
#print("Solution with Amazon Braket")
#solution = solve_bqm_in_amazon_braket(bqm)
#print_solution(solution, tree, optimal_weight)

#print("Solution with D-wave Leap")
#solve_bqm_in_leap(bqm)

#print("Solution with simulated annealing")
#solve_with_simulated_annealing(bqm)

#print("Exact solution (takes time)")
#solve_exactly()

### Run the whole algorithm using the update function

To make the problem and solution less non-trivial, we include the time component in the algorithm. One time step means that we have executed a single work on some of the data centers. At each time step, we check the current situation how sustainable way the data centers are running. For example, weather conditions (wind and amount of water in rivers, etc.) affect the production of green energy, and the data center's machines' characteristics determine part of the emissions. In real-life cases, the other workloads affect the decision, and we might need to switch to another data center. This demonstration possibly modifies these conditions more than they vary in real-life but this demonstrates better the idea of the algorithm.

Initially we proposed the following functions for the updates. At this point we learned an interesting lesson. After running the updates we noticed that it is not trivial to update QUBOs afterwards since the dependencies between variables and their coefficients form complicated structures. Thus the algorithm works best if the QUBO is always constructed from the beginning. Anyway, it would be interesting to study update methods deeper. For example, we could construct a small "difference QUBO" and merge this into the orginal QUBO.

In [18]:
def optimize_entire_workflow(solve_classical_optimal = True):
    optimal_solution_quantum = []
    optimal_weights_quantum = []
    optimal_solution_networkx = []
    optimal_weights_networkx = []
    error_statistics = []
   
    # For each work in workload, find the current optimum data center
    # After finding the data center, update the model to match with changed emission situation
    for work in workload[:-1]:
        
        # Initialize BQM
        vartype = dimod.BINARY
        bqm = dimod.BinaryQuadraticModel({}, {}, 0.0, vartype)
        variables, work_order = construct_variables(int(work["work_id"]))
        bqm = construct_bqm_constraint1(bqm, variables)
        bqm, tree = construct_bqm_constraint2(bqm, variables, work_order)
        
        # Solve the problem classically in order to evaluate the algorihtm
        if solve_classical_optimal:
            best_solution, optimal_weight = solve_with_networkx(tree, variables, str(work['work_id']))
            optimal_weights_networkx.append(optimal_weight)
            print("Best solution with networkx: ", best_solution[0])
            optimal_solution_networkx.append(best_solution[0])
        
        # Solve the problem on Amazon Braket
        solution_quantum = solve_bqm_in_amazon_braket(bqm)
        print("Optimal solution found by Amazon Braket:")
        print_solution(solution_quantum, tree, optimal_weight = optimal_weights_networkx[-1])
        
        # Calculate how much the quantum solution differs from the optimal
        positive_solution = []
        for varname, value in solution_quantum.items():
            if value == 1:
                positive_solution.append(varname)
        optimal_solution_quantum.append(positive_solution[0])
        #print("Positive solution ", positive_solution)
        
        quantum_solution_weight = 0
        for i in range(len(positive_solution) - 1):
            edge_weight = tree.get_edge_data(positive_solution[i], positive_solution[i+1])
            quantum_solution_weight += edge_weight["weight"]
        optimal_weights_quantum.append(quantum_solution_weight)
    
    return optimal_solution_quantum, optimal_weights_quantum, optimal_solution_networkx, optimal_weights_networkx, error_statistics

Finally we run the whole algorithm for the imported workloads and cloud partners.

In [19]:
#optimize_entire_workflow()

Best solution with networkx:  ('0', '20')
Optimal solution found by Amazon Braket:
('0', '20') 1
('1', '20') 1
('2', '00') 1
('3', '40') 1
('4', '20') 1
('5', '30') 1
Difference from the optimal  0.01494087005158169
Best solution with networkx:  ('1', '30')
Optimal solution found by Amazon Braket:
('1', '20') 1
('2', '30') 1
('3', '10') 1
('4', '00') 1
('5', '30') 1
Difference from the optimal  0.023285989589212047
Best solution with networkx:  ('2', '00')
Optimal solution found by Amazon Braket:
('2', '00') 1
('3', '20') 1
('4', '10') 1
('5', '20') 1
Difference from the optimal  0.0
Best solution with networkx:  ('3', '30')
Optimal solution found by Amazon Braket:
('3', '30') 1
('4', '30') 1
('5', '10') 1
Difference from the optimal  0.0
Best solution with networkx:  ('4', '10')
Optimal solution found by Amazon Braket:
('4', '10') 1
('5', '10') 1
Difference from the optimal  0.0


([('0', '20'), ('1', '20'), ('2', '00'), ('3', '30'), ('4', '10')],
 [33.07813309839711,
  14.37314678003243,
  13.691469817050889,
  3.1061327991413332,
  1.0047409568661196],
 [('0', '20'), ('1', '30'), ('2', '00'), ('3', '30'), ('4', '10')],
 [32.59119232898366,
  14.046070137051702,
  13.691469817050889,
  3.1061327991413332,
  1.0047409568661196],
 [])

## Chapter 2: Transfering problem to Qiskit

In this part of the code I rely on the [Qiskit Tutorials](https://qiskit.org/documentation/optimization/tutorials/index.html). I want to learn to understand the connection between Ocean implementation and Qiskit. The formulation in Qiskit enables solving the problem using Braket simulators and IBM Quantum systems.

### Importing Qiskit, Amazon Braket simulators and Amazon Braket Universal gate-model QPUs

In [20]:
from qiskit import IBMQ
from qiskit_optimization import QuadraticProgram

provider = IBMQ.load_account()
backend = provider.get_backend('ibmq_qasm_simulator')

We start by importing a smaller data sets.

In [ ]:
cloud_partners_file_path = os.path.join(os.path.dirname(notebook_path), "data/cloud_partners_small.json")
f = open(cloud_partners_file_path)
partners_root = json.load(f)
cloud_partners_small = partners_root["cloud_partners"]

workload_name = "workload2.json"
workload_file_path = os.path.join(os.path.dirname(notebook_path), "data/workloads/" + workload_name)
f = open(workload_file_path)
workload_root = json.load(f)
workload_small = workload_root["workload"]

### Transforming QUBO in Ocean to QUBO in Qiskit 

In [21]:
qp = QuadraticProgram()
